# deploy flask app to AWS EC2

## Connect to Server

#### 1. Sign up for a AWS account and Launch an EC2 instance

In [ ]:
1.select the AMI (Amazon Machine Image) that statisfy your need. I am using Ubuntu Server 16.04 LTS (HVM)
2.select instance type. I am using t2.micro which is eligible for free tier. Edit configuration if you need custom setting
3.Choose create a new key pair or use exsting key pair.If you choose to use existing key pair, make sure you have access to that .pem file.
4.Launch! 

## Server Setup

1.Follow AWS's instructions on how to connect to the instance
To access your instance:
Open an SSH client. (find out how to connect using PuTTY)
Locate your private key file (flask.pem). The wizard automatically detects the key you used to launch the instance.
Your key must not be publicly viewable for SSH to work. Use this command if needed:
chmod 400 flask.pem
Connect to your instance using its Public DNS:
ec2-54-219-158-170.us-west-1.compute.amazonaws.com
Example:
ssh -i "flask.pem" ubuntu@ec2-54-219-158-170.us-west-1.compute.amazonaws.com

2.install nginx

$ sudo apt-get update
$ sudo apt-get install nginx

Once nginx is installed, you should be able to go to your public IP or public DNS and see the nginx welcome page.

nginx-welcome-page

We will remove the default page by deleting the default file.

$ sudo rm /etc/nginx/sites-enabled/default
We will create a new config file in the sites-available folder and create a symbolic link to it in the sites-enabled folder.

$ sudo vim /etc/nginx/sites-available/example.com
This is how the config file will look:

server {
	listen 80;

	location / {
		proxy_pass http://127.0.0.1:8000/;
	}
}
This config file will tell the nginx server to listen on port 80 and pass all requests with the ‘/’ prefix to the server http://127.0.0.1:8000/ We do this because Gunicorn will run your Flask app on port 8000.

Now, we will need to create a symbolic link from the sites-enabled directory that points to the example.com config file we created.

$ sudo ln -s /etc/nginx/sites-available/example.com /etc/nginx/sites-enabled/example.com
We will need to restart our nginx web server in order for our changes to take into effect.

$ sudo service nginx restart


## Local Setup

Virtual Environment

I use miniconda to download and manage my libraries and virtual environments. Compares to pip, anaconda makes it much easier to install deep leanring dependencies like keras and tensorflow. Downside is your have to install miniconda on the server side as well which will take some time and space.

1. download miniconda from https://conda.io/miniconda.html

conda create -n myenv python=3.4

source activate myenv

Develop your app in this environment or run your existing app in this environment. if there are dependencies need, install them for this environment. Make sure the app is fully functional in this environment.

In [ ]:
conda env export > environment.yml

## Setup a Git repository on EC2 (optional)

Why ec2 git

1. add EC2 idenity to ssh authenticicationThis prevents problems with git later, namely getting the error “Permission denied (publickey).”

ssh-add path/to/privateEC2key.pem

create the git repository on the EC2 instance if you are already on EC2

mkdir the_project.git 
cd the_project.git 
git init --bare

Back in your local machine. Set up the local repository with your flask projects and all your files.

cd the_project 
git init git add . 
git commit -m "Initial git commit message" 
git remote add origin username@hostname.com:the_project.git 
git config --global remote.origin.receivepack "git receive-pack" 
git push origin master

Then you can use git clone the remote repository from everywhere

git clone username@hostname.com:the_project.git

## App Deployment

On the EC2 server side, you could create a new local repository by git clone or git pull from the remote repository

We have to install miniconda and create an new environment from the .yml file. This will install all the dependencies.

conda env create -f environment.yml

Once you are in your virtual env, you should be able to run your flask app just as you run it in local machine.

In your flask app.py, please the server port to http://127.0.0.1:8000/ if otherwise.

### Notes

In such case, the following error message appears:

A line showing packet_write_wait: Connection to XXX : Broken pipe 
Solution:

On the host, add those lines in the file .ssh/config

Host *
  ServerAliveInterval 30
  ServerAliveCountMax 5
If the file config does not exist, just create it.

# Resources

https://chrisdtran.com/2017/deploy-flask-on-ec2/
https://shirtdev.wordpress.com/2011/05/04/setting-up-a-git-repository-on-an-amazon-ec2-instance/
https://conda.io/docs/user-guide/tasks/manage-environments.html
http://thomas-cokelaer.info/blog/2017/05/packet_write_wait-connection-to-xxx-broken-pipe/